In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Define the URLs of the pages we will be scraping

In [37]:
dma_url = "https://www.lse.ac.uk/Mathematics/Research/Discrete-Mathematics-and-Algorithms"
fin_url = "https://www.lse.ac.uk/Mathematics/Research/Financial-Mathematics-and-Control-Theory"
game_url = "https://www.lse.ac.uk/Mathematics/Research/Game-Theory"
or_url = "https://www.lse.ac.uk/Mathematics/Research/Operations-Research"

### Create a function to help us scrape a url and return a list of professors
* `DMA` : Discrete Maths and Algorithms
* `FIN` : Financial Maths
* `GAME` : Game Theory
* `OR` : Operational Research

In [36]:
def get_professors(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    div = soup.find('div', class_ = "pageContent pageContent--std")
    # Find all headers
    profs = div.find_all('a')
    #return profs
    
    # Extract the text from each header
    profs = [prof.text for prof in profs]
    #return profs
    # Remove the `\xa0` unicode
    profs = [prof.replace('\xa0', ' ') for prof in profs]
    
    # Remove whitespace
    profs = [prof.strip() for prof in profs]
    
    return profs

### Create the 4 lists of professors

In [108]:
dma_profs = get_professors(dma_url)
fin_profs = get_professors(fin_url)
game_profs = get_professors(game_url)
or_profs = get_professors(or_url)

### The lists still require some cleaning

In [109]:
# Clean DMA list
dma_profs.pop(5)
dma_profs = dma_profs[:-8]

In [110]:
# Clean FIN list
fin_profs.pop(0)
fin_profs = fin_profs[:-9]

In [111]:
# Clean GAME list
game_profs.pop(2)
game_profs.pop(3)
game_profs.pop(4)
game_profs = game_profs[:-5]

In [112]:
# Clean OR list
for i in range(6):
    or_profs.pop(1)
or_profs = or_profs[:-8]

### Create a DataFrame
* We will have the professor as a column name
* Then their interests underneath

In [113]:
column_names = dma_profs + fin_profs + game_profs + or_profs

In [114]:
df = pd.DataFrame(columns = column_names)

In [115]:
df 

,Professor Peter Allen,Professor Martin Anthony,Dr Tugkan Batu,Professor Julia Böttcher,Professor Graham Brightwell,Professor Jan van den Heuvel,Professor Andrew Lewis-Pye,Dr Emilio Pierro,Dr Robert Simon,Professor Jozef Skokan,...,Dr Robert Simon,Professor Bernhard von Stengel,Dr Ahmad Abdi,Dr Grammateia Kotsialou,Dr Neil Olver,Dr Katerina Papadaki,Professor Gregory Sorkin,Professor László Végh,Dr Aled Williams,Dr Giacomo Zambelli
